Utilize the dataset that is provided to make a NBA predicition.

import python package

In [48]:
import pandas as pd
import math
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
import random

Init some parameters

In [49]:
base_elo = 1600
team_elos = {}
team_stats = {}
X = []
y = []
folder = 'data'

define some helper function

In [50]:
# calculate Elo score
def calc_elo(win_team, lose_team):
    winner_rank = get_elo(win_team)
    loser_rank = get_elo(lose_team)

    rank_diff = winner_rank - loser_rank
    exp = (rank_diff * -1) / 400
    odds = 1 / (1 + math.pow(10, exp))
    if winner_rank < 2100:
        k = 32
    elif 2100 <= winner_rank < 2400:
        k = 24
    else:
        k = 16
    new_winner_rank = round(winner_rank + (k * (1 - odds)))
    new_rank_diff = new_winner_rank - winner_rank
    new_loser_rank = loser_rank - new_rank_diff

    return new_winner_rank, new_loser_rank

In [51]:
# read csv file and build information matrix
def initialize_data(Mstat, Ostat, Tstat):
    new_Mstat = Mstat.drop(['Rk', 'Arena'], axis=1)
    new_Ostat = Ostat.drop(['Rk', 'G', 'MP'], axis=1)
    new_Tstat = Tstat.drop(['Rk', 'G', 'MP'], axis=1)

    team_stats1 = pd.merge(new_Mstat, new_Ostat, how='left', on='Team')
    team_stats1 = pd.merge(team_stats1, new_Tstat, how='left', on='Team')

    # print(team_stats1.info())
    return team_stats1.set_index('Team', inplace=False, drop=True)

In [52]:
def get_elo(team):
    try:
        return team_elos[team]
    except:
        # if not, init elo score as base_elo
        team_elos[team] = base_elo
        return team_elos[team]

In [57]:
def build_dataSet(team_stats, all_data):
    # print("Building data set..")
    for index, row in all_data.iterrows():
        WLoc = ''
        if float(row['PTS']) > float(row['PTS.1']):
            Wteam = row['Visitor/Neutral']
            Lteam = row['Home/Neutral']
            WLoc = 'V'
        else:
            Wteam = row['Home/Neutral']
            Lteam = row['Visitor/Neutral']
            WLoc = 'H'

        team1_elo = get_elo(Wteam)
        team2_elo = get_elo(Lteam)

        if WLoc == 'H':
            team1_elo += 100
        else:
            team2_elo += 100

        team1_features = [team1_elo]
        team2_features = [team2_elo]

        for key, value in team_stats.loc[Wteam].iteritems():
            team1_features.append(value)
        for key, value in team_stats.loc[Lteam].iteritems():
            team2_features.append(value)

        if WLoc=='H':
            X.append(team1_features + team2_features)
            y.append(1)
        else:
            X.append(team2_features + team1_features)
            y.append(0)

        new_winner_rank, new_loser_rank = calc_elo(Wteam, Lteam)
        team_elos[Wteam] = new_winner_rank
        team_elos[Lteam] = new_loser_rank

    return np.nan_to_num(X), np.array(y)

load features' files. In this sample, information of 2018-19 season is loaded. If info of other season should be loaded, just modify `18-19` to the season you want, such as `15-16`, `16-17` or `17-18`. But .csv file should be storaged in `folder`(./data).

In [58]:
Mstat = pd.read_csv(folder + '/18-19Miscellaneous_Stat.csv')
Ostat = pd.read_csv(folder + '/18-19Opponent_Per_Game_Stat.csv')
Tstat = pd.read_csv(folder + '/18-19Team_Per_Game_Stat.csv')

re-format features to train

In [59]:
team_stats = initialize_data(Mstat, Ostat, Tstat)

Load match infos as label. In this case, trainning set and test set of match result is collected from `Year_2018_2019.csv`. Files,`Year_2016_2017.csv` and `Year_2017_2018.csv`, are open to those code by shift code of `result_data = pd.read_csv(folder + '/Year_2018_2019.csv')` to:

**result_data = pd.read_csv(folder + '/Year_2016_2017.csv')** or 

**result_data = pd.read_csv(folder + '/Year_2017_2018.csv')**

In [60]:
result_data = pd.read_csv(folder + '/Year_2018_2019.csv')
X, y = build_dataSet(team_stats, result_data)

Let's Check information about features and labels.

In [75]:
print('X.shape=',X.shape)
print('Which means there are {} samples, each sample has {} features.'.format(X.shape[0],X.shape[1]))
print('such as:Feature')
print(pd.DataFrame(X).head(10))
print('lables:(1 denotes HomeWin, 0 denotes VisitorWin)')
print(pd.DataFrame(y).head(10))

X.shape= (1312, 136)
Which means there are 1312 samples, each sample has 136 features.
such as:Feature
      0     1     2     3     4     5     6     7     8      9    ...    \
0  1683.0  26.2  36.0  46.0  37.0  45.0 -1.50  0.48 -1.02  111.4  ...     
1  1821.0  24.9  54.0  28.0  51.0  31.0  3.95  0.24  4.19  113.0  ...     
2  1737.0  25.7  49.0  33.0  52.0  30.0  4.44 -0.54  3.90  112.2  ...     
3  1941.0  28.4  57.0  25.0  56.0  26.0  6.46 -0.04  6.42  115.9  ...     
4  1656.0  25.7  33.0  49.0  38.0  44.0 -1.33  0.23 -1.10  111.4  ...     
5  1724.0  26.2  36.0  46.0  37.0  45.0 -1.50  0.48 -1.02  111.4  ...     
6  1707.0  26.9  33.0  49.0  38.0  44.0 -1.28  0.42 -0.87  109.4  ...     
7  1734.0  25.7  49.0  33.0  50.0  32.0  3.40  0.15  3.56  110.3  ...     
8  1883.0  29.2  53.0  29.0  53.0  29.0  4.77  0.19  4.96  115.5  ...     
9  1655.0  25.7  49.0  33.0  50.0  32.0  3.40  0.15  3.56  110.3  ...     

     126   127   128   129   130  131  132   133   134    135  
0  0.77

And normally in machine learning, data sets should be shuffled.

In [62]:
per = np.random.permutation(len(y))
X = X[per, :]
y = y[per]

Split data sets to training set and test set. 80% of data should be for training, and remainder for testing.

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print('sample of trainning set:{},sample of test set:{}'.format(len(y_train), len(y_test)))

sample of trainning set:1049,sample of test set:263


**Build Decision Tree Random Forest XGBoost Logistic Regression Naive Bayes (Gaussian)**

Build and train **LogisticRegression** model

In [64]:
lr = LogisticRegression(penalty='l2', tol=0.00001, C=0.5, solver='liblinear')
lr.fit(X_train, y_train)
score_lr = lr.score(X_test, y_test)
score_lr_train = lr.score(X_train, y_train)
print('score of Logistic Regression model in test set is {:.4f} (score of training set:{:.4f})'.format(score_lr,score_lr_train))

score of Logistic Regression model in test set is 0.7034 (score of training set:0.6997)


Build and train **GaussianNB** model. 
more setting:https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html

In [65]:
gnb = GaussianNB(var_smoothing=1e-09)
gnb.fit(X_train, y_train)
score_gnb = gnb.score(X_test, y_test)
score_gnb_train = gnb.score(X_train, y_train)
print('score of Naive Bayes (Gaussian) model in test set is {:.4f} (score of training set:{:.4f})'.format(score_gnb,score_gnb_train))

score of Naive Bayes (Gaussian) model in test set is 0.6654 (score of training set:0.6692)


Build and train **Random Forest** model.
more setting:https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [66]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
score_rf = rf.score(X_test, y_test)
score_rf_train = rf.score(X_train, y_train)
print('score of Random Forest model in test set is {:.4f} (score of training set:{:.4f})'.format(score_rf,score_rf_train))

score of Random Forest model in test set is 0.6122 (score of training set:0.9762)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Build and train **Decision Tree** model.
more setting: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

In [67]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
score_dt = dt.score(X_test, y_test)
score_dt_train = dt.score(X_train, y_train)
print('score of Decision Tree model in test set is {:.4f} (score of training set:{:.4f})'.format(score_dt,score_dt_train))

score of Decision Tree model in test set is 0.5513 (score of training set:0.9990)


Build and train **XGBoost** model. 
more setting: https://dask-ml.readthedocs.io/en/stable/modules/generated/dask_ml.xgboost.XGBClassifier.html

In [68]:
xgb = XGBClassifier(learning_rate=0.01, max_depth=5)
xgb.fit(X_train, y_train)
score_xgb = xgb.score(X_test, y_test)
score_xgb_train = xgb.score(X_train, y_train)
print('score of XGBoost model in test set is {:.4f} (score of training set:{:.4f})'.format(score_xgb,score_xgb_train))

score of XGBoost model in test set is 0.6236 (score of training set:0.8046)


For summery info:

In [69]:
print('score of Logistic Regression model in test set is {:.4f} (score of training set:{:.4f})'.format(score_lr,score_lr_train))
print('score of Naive Bayes (Gaussian) model in test set is {:.4f} (score of training set:{:.4f})'.format(score_gnb,score_gnb_train))
print('score of Random Forest model in test set is {:.4f} (score of training set:{:.4f})'.format(score_rf,score_rf_train))
print('score of Decision Tree model in test set is {:.4f} (score of training set:{:.4f})'.format(score_dt,score_dt_train))
print('score of XGBoost model in test set is {:.4f} (score of training set:{:.4f})'.format(score_xgb,score_xgb_train))

score of Logistic Regression model in test set is 0.7034 (score of training set:0.6997)
score of Naive Bayes (Gaussian) model in test set is 0.6654 (score of training set:0.6692)
score of Random Forest model in test set is 0.6122 (score of training set:0.9762)
score of Decision Tree model in test set is 0.5513 (score of training set:0.9990)
score of XGBoost model in test set is 0.6236 (score of training set:0.8046)


When you wanna test the model with 2019-2020 season match information, an .csv file named `Year_2019_2020.csv` should be storaged in `data` subpath. Re-load match info with code as below.

**Note:** if you do not have `Year_2019_2020.csv`, you will get an error as **FileNotFoundError: File b'data/Year_2019_2020.csv' does not exist**

If pruning model needed, the websites are listed above for more parameter setting. For example, we create GaussianNB model with one line code `gnb = GaussianNB(var_smoothing=1e-09)`, if the [link](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html) is checked,  two parameter(priors,var_smoothing) can be set. Please feel free to modify one or two parameter(s). That will be interesting if you implyment this part by yourself. 


In [74]:
result_data = pd.read_csv(folder + '/Year_2019_2020.csv')
X_2019, y_2019 = build_dataSet(team_stats, result_data)
score_lr = lr.score(X_2019, y_2019)
score_gnb = gnb.score(X_2019, y_2019)
score_rf = rf.score(X_2019, y_2019)
score_dt = dt.score(X_2019, y_2019)
score_xgb = xgb.score(X_2019, y_2019)
print('score of Logistic Regression model in test set is {:.4f}'.format(score_lr))
print('score of Naive Bayes (Gaussian) model in test set is {:.4f}'.format(score_gnb))
print('score of Random Forest model in test set is {:.4f}'.format(score_rf))
print('score of Decision Tree model in test set is {:.4f}'.format(score_dt))
print('score of XGBoost model in test set is {:.4f}'.format(score_xgb))

FileNotFoundError: File b'data/Year_2019_2020.csv' does not exist